Simple first-order maximum entropy markov model with greedy decoding for data with span annotations in BIO notation.  This model predicts a label for every *sentence* in the input (and represents that time step as the bag of words within that sentence, along with the previous sentence tag).  This assumes the input file is one long document, only represents time step t through the words observed in sentence t (and not, e.g., the words in sentence t-1 or t+1), and uses greedy decoding (not Viterbi decoding), so lots of directions to build on.

In [1]:
import sys
from collections import Counter
import operator
import nltk
from scipy import sparse
from sklearn import linear_model

In [2]:
def read_data(filename):
    x=[]
    y=[]
    with open(filename) as file:
        for line in file:
            cols=line.rstrip().split("\t")
            if len(cols) >= 3:
                label=cols[1]
                text=cols[2]
                x.append(text)
                y.append(label)

    return x,y


In [3]:
def get_text_feats(text):
    feats={}
    tokens=nltk.word_tokenize(text.lower())
    for tok in tokens:
        feats[tok]=1
    return feats

def get_text_feats_in_vocab(text, vocab):
    all_feats=get_text_feats(text)
    feats={}
    for feat in all_feats:
        if feat in vocab:
            feats[vocab[feat]]=1
    return feats

In [4]:
def get_feature_vocab(x, y, min_feat_count=2):

    vocab=Counter()
    for i in range(len(x)):

        lastLabel="START"
        if i > 0:
            lastLabel=y[i-1]

        vocab["_y_i_1:%s" % lastLabel]+=1

        feats=get_text_feats(x[i])
        for feat in feats:
            vocab[feat]+=1

    feats={}
    for feat in vocab:
        if vocab[feat] >= min_feat_count or feat.startswith("_y_i_1"):
            feats[feat]=len(feats)

    return feats


In [5]:
def greedy_decode(model, x, vocab):
    # with greedy decoding, we make decision sequentially and commit to each one in turn.
    preds=[]
    prev_lab="START"
    for i in range(len(x)):
        feats=get_text_feats_in_vocab(x[i], vocab)
        # prev_lab here is the *prediction* for the previous time step.  For dev and test, we assume we don't
        # have access to the true labels at those positions, so much rely on the predictions our model makes.
        feats[vocab["_y_i_1:%s" % prev_lab]]=1
        X = sparse.dok_matrix((1, len(vocab)))
        for fid in feats:
            X[0,fid]=1
        y=model.predict(X)[0]
        preds.append(y)
        prev_lab=y

    return preds

In [6]:
def featurize(x, y, feats):
    F = len(feats)
    D = len(x)
    X = sparse.dok_matrix((D, F))
    Y = []

    for i in range(len(x)):

        lastLabel="START"
        if i > 0:
            lastLabel=y[i-1]
        feat="_y_i_1:%s" % lastLabel
        X[i,feats[feat]]=1

        this_feats=get_text_feats_in_vocab(x[i], feats)
        for feat in this_feats:
            X[i,feat]=1

        Y.append(y[i])

    return X, Y

In [7]:
def get_spans(labels):

    spans=[]
    start=None

    for idx, label in enumerate(labels):

        parts=label.split("-")
        bio=cat="O"
        if len(parts) == 2:
            bio=parts[0]
            cat=parts[1]
        else:
            bio=parts[0]

        if (bio == "B" or bio == "O") and start is not None:
            s, last_cat=start
            spans.append((idx-1,last_cat))
            start=None

        if bio == "B":
            start=idx, cat

    if start is not None:
        s, last_cat=start
        spans.append((len(labels)-1,last_cat))

    return set(spans)

In [8]:
def get_span_f1(pred_labels, true_labels):

    cor=0
    precision_n=0
    recall_n=0

    pred_spans=get_spans(pred_labels)
    true_spans=get_spans(true_labels)


    cor=len(pred_spans.intersection(true_spans))
    precision_n=len(pred_spans)
    recall_n=len(true_spans)
        
    precision=0
    if precision_n > 0:
        precision=cor/precision_n

    recall=0
    if recall_n > 0:
        recall=cor/recall_n
    F1=0
    if precision+recall > 0:
        F1=2*precision*recall/(precision+recall)

    return F1


In [19]:
def printWeights(log_reg, feature_vocab, n=10):

    reverse_vocab=[None]*len(log_reg.coef_[0])
    for k in feature_vocab:
        reverse_vocab[feature_vocab[k]]=k

    # binary
    if len(log_reg.classes_) == 2:
        weights=log_reg.coef_[0]

        cat=log_reg.classes_[1]
        for feature, weight in list(reversed(sorted(zip(reverse_vocab, weights), key = operator.itemgetter(1))))[:n]:
            print("%s\t%.3f\t%s" % (cat, weight, feature))
        print()

        cat=log_reg.classes_[0]
        for feature, weight in list(sorted(zip(reverse_vocab, weights), key = operator.itemgetter(1)))[:n]:
            print("%s\t%.3f\t%s" % (cat, weight, feature))
        print()

    # multiclass
    else:
        for i, cat in enumerate(log_reg.classes_):

            weights=log_reg.coef_[i]

            for feature, weight in list(reversed(sorted(zip(reverse_vocab, weights), key = operator.itemgetter(1))))[:n]:
                print("%s\t%.3f\t%s" % (cat, weight, feature))
            print("")

In [23]:
gid=20
folder="splits/%s" % gid
trainFile="%s/train.txt" % folder
devFile="%s/dev.txt" % folder
testFile="%s/test.txt" % folder

train_x, train_y=read_data(trainFile)
dev_x, dev_y=read_data(devFile)
test_x, test_y=read_data(testFile)

In [24]:
vocab=get_feature_vocab(train_x, train_y)
train_X_ids, train_Y_ids=featurize(train_x, train_y, vocab)

best_dev_F1=0
best_model=None

for C in [0.1, 1, 10, 100]:
    log_reg = linear_model.LogisticRegression(C = C, max_iter=1000)
    log_reg.fit(train_X_ids, train_Y_ids)

    preds=greedy_decode(log_reg, dev_x, vocab)
    dev_F1=get_span_f1(preds, dev_y)
    if dev_F1 > best_dev_F1:
        best_dev_F1=dev_F1
        best_model=log_reg

log_reg=best_model

preds=greedy_decode(log_reg, test_x, vocab)
test_F1=get_span_f1(preds, test_y)
print("Test Span F1 for best dev model: %.3f\n" % test_F1)



Test Span F1 for best dev model: 0.341



In [25]:
printWeights(log_reg, vocab)

B	4.173	murdered
B	3.479	dies
B	3.383	killed
B	3.366	unfortunately
B	3.307	actually
B	3.247	murders
B	3.166	vader
B	3.129	mountain
B	3.081	revealed
B	3.058	suddenly

I	3.706	wounded
I	3.093	awakens
I	2.895	_y_i_1:B
I	2.834	person
I	2.815	sex
I	2.499	room
I	2.305	maureen
I	2.153	fall
I	2.082	temüjin
I	2.073	before

O	3.538	game
O	3.263	whom
O	3.201	gawda
O	2.909	upon
O	2.889	various
O	2.642	date
O	2.449	rothbaum
O	2.392	temple
O	2.346	_y_i_1:O
O	2.330	plans

